# Feature Engineering y data enrichment

## Enriquecimiento del dataset

### Descargar datos desde api 

Instalamos librerías necesarias para Open-Meteo

In [3]:
!pip install openmeteo-requests requests-cache retry-requests

   ---------------------------------------- 0.0/670.9 kB ? eta -:--:--
   ---------------------------------------- 670.9/670.9 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 4.2 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
Importamos librerias y datos necesarios

In [1]:
import pandas as pd

# --- 1. Carga del Dataset Base ---
ruta_dataset = "../data/processed/dataset_final_etiquetado.csv"
df = pd.read_csv(ruta_dataset)
df['date'] = pd.to_datetime(df['date']) # Muy importante para manipular fechas

print("✅ Dataset base cargado exitosamente.")
print(f"Dimensiones iniciales: {df.shape}")

# --- 2. Ingeniería de Características (Feature Engineering) ---

# --- Características Temporales ---
df['año'] = df['date'].dt.year
df['mes'] = df['date'].dt.month
df['dia_del_año'] = df['date'].dt.dayofyear # El número de día del 1 al 365

# --- Característica de Interacción ---
df['rango_temp_diario'] = df['TMAX'] - df['TMIN']

print("\n✅ Nuevas características creadas:")
# Mostramos el resultado con las nuevas columnas al final
display(df[['date', 'año', 'mes', 'dia_del_año', 'rango_temp_diario', 'granizo']].head())

✅ Dataset base cargado exitosamente.
Dimensiones iniciales: (37399, 8)

✅ Nuevas características creadas:


,date,año,mes,dia_del_año,rango_temp_diario,granizo
0,2000-01-01,2000,1,1,NaN,0
1,2000-01-01,2000,1,1,15.8,1
2,2000-01-01,2000,1,1,NaN,0
3,2000-01-01,2000,1,1,15.8,0
4,2000-01-02,2000,1,2,17.5,0


Cargamos el dataset y mostramos unas nuevas características agregadas, día del año y rango de temperatura diario.
La intención es tener la mayor cantidad de datos disponibles para un análisis más exhaustivo y ver que patrones podemos encontrar.

Veremos el resultado del nuevo df

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37399 entries, 0 to 37398
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               37399 non-null  datetime64[ns]
 1   station_name       37399 non-null  object        
 2   PRCP               9668 non-null   float64       
 3   SNWD               225 non-null    float64       
 4   TAVG               37397 non-null  float64       
 5   TMAX               19599 non-null  float64       
 6   TMIN               35148 non-null  float64       
 7   granizo            37399 non-null  int64         
 8   año                37399 non-null  int32         
 9   mes                37399 non-null  int32         
 10  dia_del_año        37399 non-null  int32         
 11  rango_temp_diario  18324 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int32(3), int64(1), object(1)
memory usage: 3.0+ MB


Prueba de api - Comparación de estaciones vs. obtenidas por API

In [7]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# --- 1. Configuración ---
# (Asumimos que el cliente de Open-Meteo ya está configurado en una celda anterior)
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# --- 2. Nuestros datos de estaciones (ahora con la elevación conocida) ---
stations_info_data = {
    'station_name': [
        'MENDOZA AERO, AR', 'SAN MARTIN, AR', 'MENDOZA OBSERVATORIO, AR', 
        'MALARGUE, AR', 'SAN RAFAEL, AR'
    ],
    'latitude': [-32.83, -33.08, -32.9, -35.48, -34.58],
    'longitude': [-68.78, -68.48, -68.87, -69.58, -68.33],
    'elevation_conocida': [704, 653, 827, 1425, 748] # Elevación en metros
}
stations_info_df = pd.DataFrame(stations_info_data)

print("--- Verificando Coordenadas con la API de Open-Meteo ---")
print("Estación                       | Elevación Conocida | Elevación API (Modelo)")
print("-------------------------------|--------------------|------------------------")

# --- 3. Bucle de Verificación ---
for index, station in stations_info_df.iterrows():
    params = {
        "latitude": station['latitude'],
        "longitude": station['longitude'],
        "daily": "temperature_2m_max", # Pedimos cualquier variable para obtener la metadata
        "start_date": "2024-01-01",
        "end_date": "2024-01-01"
    }
    
    try:
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
        response = responses[0]
        
        # Obtenemos la elevación de la respuesta de la API
        elevation_api = response.Elevation()
        
        # Imprimimos la comparación
        print(f"{station['station_name']:<30} | {station['elevation_conocida']:>18}m | {elevation_api:.2f}m")
        
    except Exception as e:
        print(f"Error al verificar {station['station_name']}: {e}")

--- Verificando Coordenadas con la API de Open-Meteo ---
Estación                       | Elevación Conocida | Elevación API (Modelo)
-------------------------------|--------------------|------------------------
MENDOZA AERO, AR               |                704m | 701.00m
SAN MARTIN, AR                 |                653m | 661.00m
MENDOZA OBSERVATORIO, AR       |                827m | 830.00m
MALARGUE, AR                   |               1425m | 1418.00m
SAN RAFAEL, AR                 |                748m | 691.00m


Descarga unitaria de estaciones(Para descargar en su pc, solo correr esto, lo anterior fue de prueba y conocimiento de errores)

In [32]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import time
import os
import calendar

# --- 1. Definimos Nuestras Estaciones Objetivo ---
stations_info_data = {
    'station_name': ['MENDOZA AERO, AR', 'SAN MARTIN, AR', 'MENDOZA OBSERVATORIO, AR', 'MALARGUE, AR', 'SAN RAFAEL, AR'],
    'latitude': [-32.83, -33.08, -32.9, -35.48, -34.58],
    'longitude': [-68.78, -68.48, -68.87, -69.58, -68.33]
}
stations_info_df = pd.DataFrame(stations_info_data)
print("✅ Lista de estaciones definida.")

# --- 2. Configurar la API de Open-Meteo ---
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)
print("✅ Cliente de API listo.")

# --- 3. Función de Descarga (Versión Adaptativa y Corregida) ---
def descargar_openmeteo_estacion(station_info):
    lat, lon = station_info['latitude'], station_info['longitude']
    start_year, end_year = 2000, 2024
    
    lista_df_obtenidos = []
    print(f"  -> Procesando años desde {start_year} hasta {end_year}...")
    
    for year in range(start_year, end_year + 1):
        daily_vars = ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_mean", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration", "dew_point_2m_mean", "relative_humidity_2m_mean", "pressure_msl_mean"]
        try:
            # Intento 1: Pedir el año completo
            params_anual = {"latitude": lat, "longitude": lon, "start_date": f"{year}-01-01", "end_date": f"{year}-12-31", "daily": daily_vars, "timezone": "auto"}
            responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params_anual, timeout=120)
            response = responses[0]
            
            daily = response.Daily()
            daily_data = {"date": pd.date_range(start=pd.to_datetime(daily.Time(), unit="s"), end=pd.to_datetime(daily.TimeEnd(), unit="s"), freq=pd.Timedelta(seconds=daily.Interval()), inclusive="left")}
            for i, var in enumerate(daily_vars): daily_data[f"om_{var}"] = daily.Variables(i).ValuesAsNumpy()
            
            lista_df_obtenidos.append(pd.DataFrame(data=daily_data))
            print(f"    -> Datos para el año {year} obtenidos de una vez.")
            time.sleep(5)
        except Exception as e_year:
            # Si falla el intento anual, reintentamos mes por mes
            print(f"    -> ERROR al obtener el año {year} completo. Reintentando mes por mes...")
            for month in range(1, 13):
                try:
                    ultimo_dia = calendar.monthrange(year, month)[1]
                    params_mensual = {"latitude": lat, "longitude": lon, "start_date": f"{year}-{month:02d}-01", "end_date": f"{year}-{month:02d}-{ultimo_dia}", "daily": daily_vars, "timezone": "auto"}
                    responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params_mensual, timeout=60)
                    response = responses[0]

                    daily = response.Daily()
                    daily_data = {"date": pd.date_range(start=pd.to_datetime(daily.Time(), unit="s"), end=pd.to_datetime(daily.TimeEnd(), unit="s"), freq=pd.Timedelta(seconds=daily.Interval()), inclusive="left")}
                    for i, var in enumerate(daily_vars): daily_data[f"om_{var}"] = daily.Variables(i).ValuesAsNumpy()
                    
                    lista_df_obtenidos.append(pd.DataFrame(data=daily_data))
                    print(f"      -> Mes {month}/{year} OK.")
                    time.sleep(2)
                except Exception as e_month:
                    print(f"      -> ERROR en mes {month}/{year}. Error: {e_month}")

    if lista_df_obtenidos:
        return pd.concat(lista_df_obtenidos, ignore_index=True)
    else:
        return pd.DataFrame()

# --- 4. Bucle Principal de Descarga ---
print("\nIniciando descarga de datos de Open-Meteo...")
os.makedirs("../data/raw/openmeteo/", exist_ok=True) 

for index, station in stations_info_df.iterrows():
    station_name = station['station_name']
    nombre_archivo_checkpoint = f"../data/raw/openmeteo/{station_name.replace(', AR', '').replace(' ', '_')}.csv"
    
    if os.path.exists(nombre_archivo_checkpoint):
        print(f"\nCheckpoint encontrado para '{station_name}'. Saltando.")
        continue

    print(f"\nDescargando datos para '{station_name}'...")
    df_openmeteo = descargar_openmeteo_estacion(station)
    
    if not df_openmeteo.empty:
        df_openmeteo['station_name'] = station_name
        df_openmeteo['latitude'] = station['latitude']
        df_openmeteo['longitude'] = station['longitude']
        df_openmeteo.to_csv(nombre_archivo_checkpoint, index=False)
        print(f"✅ Checkpoint guardado para '{station_name}'")

✅ Lista de estaciones definida.
✅ Cliente de API listo.

Iniciando descarga de datos de Open-Meteo...

Descargando datos para 'MENDOZA AERO, AR'...
  -> Procesando años desde 2000 hasta 2024...
    -> Datos para el año 2000 obtenidos de una vez.
    -> Datos para el año 2001 obtenidos de una vez.
    -> Datos para el año 2002 obtenidos de una vez.
    -> Datos para el año 2003 obtenidos de una vez.
    -> Datos para el año 2004 obtenidos de una vez.
    -> Datos para el año 2005 obtenidos de una vez.
    -> Datos para el año 2006 obtenidos de una vez.
    -> Datos para el año 2007 obtenidos de una vez.
    -> Datos para el año 2008 obtenidos de una vez.
    -> Datos para el año 2009 obtenidos de una vez.
    -> Datos para el año 2010 obtenidos de una vez.
    -> Datos para el año 2011 obtenidos de una vez.
    -> Datos para el año 2012 obtenidos de una vez.
    -> Datos para el año 2013 obtenidos de una vez.
    -> Datos para el año 2014 obtenidos de una vez.
    -> Datos para el año 2

### Unión, limpieza y transformación de datos

Transformacion y Limpieza

Luego de ver que sucedieron errores en la descarga y luego la unión se descarga nuevamente los datos, pero solamente por estación sin unir o demás.

Primero se realzira una verificacion de fechas por bugs anteriormente ocurridos

In [33]:
import pandas as pd

# --- 1. Seleccionamos un archivo para verificar ---
# Podés cambiar el nombre del archivo para revisar otra de las estaciones
ruta_archivo_checkpoint = "../data/raw/openmeteo/MENDOZA_AERO.csv"

print(f"--- Verificando el archivo: {ruta_archivo_checkpoint} ---")

try:
    # --- 2. Cargamos el archivo ---
    df_verificacion = pd.read_csv(ruta_archivo_checkpoint)
    # Convertimos la columna 'date' a formato de fecha para analizarla
    df_verificacion['date'] = pd.to_datetime(df_verificacion['date'])
    
    # --- 3. Hacemos las verificaciones ---
    fecha_minima = df_verificacion['date'].min()
    fecha_maxima = df_verificacion['date'].max()
    total_dias = len(df_verificacion)
    
    print(f"\n✅ Primera fecha en el archivo: {fecha_minima.strftime('%Y-%m-%d')}")
    print(f"✅ Última fecha en el archivo:  {fecha_maxima.strftime('%Y-%m-%d')}")
    print(f"✅ Total de días (filas) descargados: {total_dias}")
    
    # --- 4. Muestra visual ---
    print("\n--- Primeros 5 días del archivo: ---")
    display(df_verificacion.head())
    
    print("\n--- Últimos 5 días del archivo: ---")
    display(df_verificacion.tail())

except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo en la ruta especificada.")

--- Verificando el archivo: ../data/raw/openmeteo/MENDOZA_AERO.csv ---

✅ Primera fecha en el archivo: 2000-01-01
✅ Última fecha en el archivo:  2024-12-31
✅ Total de días (filas) descargados: 9132

--- Primeros 5 días del archivo: ---


,date,om_weather_code,om_temperature_2m_max,om_temperature_2m_min,om_apparent_temperature_mean,om_precipitation_sum,om_rain_sum,om_snowfall_sum,om_precipitation_hours,om_wind_gusts_10m_max,om_wind_direction_10m_dominant,om_shortwave_radiation_sum,om_et0_fao_evapotranspiration,om_dew_point_2m_mean,om_relative_humidity_2m_mean,om_pressure_msl_mean,station_name,latitude,longitude
0,2000-01-01 03:00:00,61.0,26.667501,15.817500,24.009987,1.6,1.6,0.0,1.0,23.759998,10.304779,28.25,5.423854,16.552917,73.381630,1010.19995,"MENDOZA AERO, AR",-32.83,-68.78
1,2000-01-02 03:00:00,51.0,26.417501,16.167501,24.068575,0.2,0.2,0.0,1.0,24.119999,41.987020,29.44,5.736221,16.148746,70.359700,1010.22920,"MENDOZA AERO, AR",-32.83,-68.78
2,2000-01-03 03:00:00,61.0,24.367500,16.967500,21.537270,8.1,8.1,0.0,8.0,20.519999,159.863620,19.98,3.778040,15.994583,78.389275,1010.57495,"MENDOZA AERO, AR",-32.83,-68.78
3,2000-01-04 03:00:00,2.0,27.267500,14.417500,22.500076,0.0,0.0,0.0,0.0,27.000000,8.806673,31.88,6.212411,13.934166,65.079216,1009.19580,"MENDOZA AERO, AR",-32.83,-68.78
4,2000-01-05 03:00:00,51.0,28.467500,16.867500,23.890268,0.3,0.3,0.0,2.0,33.120000,8.009057,26.90,5.706869,14.636249,61.392567,1008.90410,"MENDOZA AERO, AR",-32.83,-68.78



--- Últimos 5 días del archivo: ---


,date,om_weather_code,om_temperature_2m_max,om_temperature_2m_min,om_apparent_temperature_mean,om_precipitation_sum,om_rain_sum,om_snowfall_sum,om_precipitation_hours,om_wind_gusts_10m_max,om_wind_direction_10m_dominant,om_shortwave_radiation_sum,om_et0_fao_evapotranspiration,om_dew_point_2m_mean,om_relative_humidity_2m_mean,om_pressure_msl_mean,station_name,latitude,longitude
9127,2024-12-27 03:00:00,3.0,36.719,21.219000,26.819666,0.0,0.0,0.0,0.0,46.079998,189.90659,24.72,7.264482,5.494000,23.920776,1009.58344,"MENDOZA AERO, AR",-32.83,-68.78
9128,2024-12-28 03:00:00,51.0,34.969,22.418999,27.443064,0.6,0.6,0.0,2.0,37.800000,217.44868,25.73,7.576168,8.969001,30.308828,1013.61237,"MENDOZA AERO, AR",-32.83,-68.78
9129,2024-12-29 03:00:00,53.0,36.269,20.769000,27.816118,1.0,1.0,0.0,3.0,53.280000,166.01964,27.34,7.301054,10.554416,35.396473,1008.64594,"MENDOZA AERO, AR",-32.83,-68.78
9130,2024-12-30 03:00:00,2.0,34.319,19.869000,27.168503,0.0,0.0,0.0,0.0,53.280000,341.15005,27.31,6.853123,12.358584,43.269135,1007.33746,"MENDOZA AERO, AR",-32.83,-68.78
9131,2024-12-31 03:00:00,1.0,33.569,22.519000,27.247261,0.0,0.0,0.0,0.0,43.560000,174.95149,28.21,7.880547,12.241917,38.796635,1006.19170,"MENDOZA AERO, AR",-32.83,-68.78


Como se observa tenemos el rango de fechas correcto.
Ya que la comprobación manual da el mismo resultado

    Periodo 2025 - 2000 = 25 años
    Año = 365 días
    365 x 25 = 9,125 días
    Valor obtenido de api = 9132

Esas leves diferencia se debe a la exactitud de la multiplicación y problemas de calendarios, pero para este análisis no afecta en nuestro modelo.

Analisis de NaN de cada Estacion 

In [34]:
import pandas as pd
import glob

# Obtenemos la lista de todos los archivos de checkpoint de Open-Meteo
ruta_checkpoints = "../data/raw/openmeteo/*.csv"
archivos_om = glob.glob(ruta_checkpoints)

print("--- Iniciando Auditoría de Datos Faltantes en Archivos de Open-Meteo ---")

# Creamos un diccionario para guardar un resumen de los hallazgos
resumen_auditoria = {}

for archivo in sorted(archivos_om):
    station_name = archivo.split('/')[-1].replace('.csv', '').replace('enriched_', '')
    print(f"\n\n--- Analizando Archivo: {station_name} ---")
    
    df_station = pd.read_csv(archivo, parse_dates=['date'])
    
    # --- Verificación 1: Días Faltantes en la Secuencia ---
    # Creamos un rango de fechas perfecto desde el inicio hasta el fin
    date_range_esperado = pd.date_range(start=df_station['date'].min(), end=df_station['date'].max())
    # Comparamos el rango perfecto con las fechas que realmente tenemos
    dias_faltantes = date_range_esperado.difference(df_station['date'])
    
    if dias_faltantes.empty:
        print("✅ Verificación de secuencia: No faltan días en el rango de fechas.")
        resumen_auditoria[station_name] = {'dias_faltantes': 0, 'filas_con_nan': 0, 'años_afectados_nan': []}
    else:
        print(f"❌ ¡Atención! Faltan {len(dias_faltantes)} días en la secuencia.")
        resumen_auditoria[station_name] = {'dias_faltantes': len(dias_faltantes), 'filas_con_nan': 0, 'años_afectados_nan': []}

    # --- Verificación 2: Valores Nulos (NaN) en los Datos Existentes ---
    # Buscamos filas que tengan al menos un valor nulo en cualquier columna
    filas_con_nan = df_station[df_station.isnull().any(axis=1)]

    if filas_con_nan.empty:
        print("✅ Verificación de nulos: No se encontraron valores NaN en ninguna columna.")
    else:
        print(f"❌ ¡Atención! Se encontraron {len(filas_con_nan)} filas con al menos un valor NaN.")
        # Extraemos los años con problemas para nuestro reporte
        años_con_nan = sorted(filas_con_nan['date'].dt.year.unique())
        print(f"   -> Años afectados por NaN: {años_con_nan}")
        resumen_auditoria[station_name]['filas_con_nan'] = len(filas_con_nan)
        resumen_auditoria[station_name]['años_afectados_nan'] = años_con_nan

# --- Imprimir Resumen Final ---
print("\n\n--- RESUMEN FINAL DE LA AUDITORÍA ---")
df_resumen = pd.DataFrame.from_dict(resumen_auditoria, orient='index')
display(df_resumen)

--- Iniciando Auditoría de Datos Faltantes en Archivos de Open-Meteo ---


--- Analizando Archivo: openmeteo\MALARGUE ---
✅ Verificación de secuencia: No faltan días en el rango de fechas.
✅ Verificación de nulos: No se encontraron valores NaN en ninguna columna.


--- Analizando Archivo: openmeteo\MENDOZA_AERO ---
✅ Verificación de secuencia: No faltan días en el rango de fechas.
✅ Verificación de nulos: No se encontraron valores NaN en ninguna columna.


--- Analizando Archivo: openmeteo\MENDOZA_OBSERVATORIO ---
✅ Verificación de secuencia: No faltan días en el rango de fechas.
✅ Verificación de nulos: No se encontraron valores NaN en ninguna columna.


--- Analizando Archivo: openmeteo\SAN_MARTIN ---
❌ ¡Atención! Faltan 61 días en la secuencia.
✅ Verificación de nulos: No se encontraron valores NaN en ninguna columna.


--- Analizando Archivo: openmeteo\SAN_RAFAEL ---
❌ ¡Atención! Faltan 92 días en la secuencia.
✅ Verificación de nulos: No se encontraron valores NaN en ninguna colum

,dias_faltantes,filas_con_nan,años_afectados_nan
openmeteo\MALARGUE,0,0,[]
openmeteo\MENDOZA_AERO,0,0,[]
openmeteo\MENDOZA_OBSERVATORIO,0,0,[]
openmeteo\SAN_MARTIN,61,0,[]
openmeteo\SAN_RAFAEL,92,0,[]


Unión de datos
En el siguiente código se unificará el archivo CSV con los días de granizo (obtenidos de NOAA, scraping, entre otras fuentes) junto con la descarga masiva de datos de Open-Meteo para cada estación. Además, se incorporarán índices adicionales con el fin de realizar un análisis más riguroso.

In [35]:
import pandas as pd
import glob
import os

print("--- Iniciando Parte 2: Unión Final de Datasets ---")

# --- 1. Cargar el Dataset Base de la NOAA ---
ruta_noaa = "../data/processed/dataset_final_etiquetado.csv"
df_noaa = pd.read_csv(ruta_noaa)
df_noaa['date'] = pd.to_datetime(df_noaa['date']).dt.date
print(f"✅ Dataset de NOAA cargado ({len(df_noaa)} filas).")

# --- 2. Cargar y Unir TODOS los archivos de Open-Meteo ---
ruta_openmeteo = "../data/raw/openmeteo/*.csv"
archivos_om = glob.glob(ruta_openmeteo)
if archivos_om:
    df_openmeteo_raw = pd.concat([pd.read_csv(f) for f in archivos_om], ignore_index=True)
    df_openmeteo_raw['date'] = pd.to_datetime(df_openmeteo_raw['date']).dt.date
    print(f"✅ {len(archivos_om)} archivos de Open-Meteo cargados y unificados.")
else:
    print("❌ No se encontraron archivos de Open-Meteo para unir.")
    df_openmeteo_raw = pd.DataFrame()

if not df_openmeteo_raw.empty:
    # --- 3. Agregar y promediar datos de Open-Meteo ---
    print("Agregando y promediando los datos de Open-Meteo por día y estación...")
    # Seleccionamos solo las columnas a promediar
    columnas_a_promediar = [col for col in df_openmeteo_raw.columns if col.startswith('om_')]
    agregaciones = {col: 'mean' for col in columnas_a_promediar}
    
    # Agrupamos por fecha y estación, manteniendo latitud y longitud
    df_openmeteo_agregado = df_openmeteo_raw.groupby(
        ['date', 'station_name', 'latitude', 'longitude']
    ).agg(agregaciones).reset_index()
    print("✅ Datos de Open-Meteo agregados.")

    # --- 4. La Unión Final ---
    df_final = pd.merge(df_noaa, df_openmeteo_agregado, on=['date', 'station_name'], how='left')
    print("\n✅ Unión final completada.")

    # --- 5. Guardar ---
    ruta_salida = "../data/processed/dataset_final_enriquecido.csv"
    df_final.to_csv(ruta_salida, index=False)
    print(f"\n✅ Dataset final enriquecido guardado en '{ruta_salida}'")
    
    print("\n--- Muestra del Dataset Final y Enriquecido ---")
    display(df_final.head())
    
    print("\n--- Información del Dataset Final ---")
    df_final.info()

--- Iniciando Parte 2: Unión Final de Datasets ---
✅ Dataset de NOAA cargado (37399 filas).
✅ 5 archivos de Open-Meteo cargados y unificados.
Agregando y promediando los datos de Open-Meteo por día y estación...
✅ Datos de Open-Meteo agregados.

✅ Unión final completada.

✅ Dataset final enriquecido guardado en '../data/processed/dataset_final_enriquecido.csv'

--- Muestra del Dataset Final y Enriquecido ---


,date,station_name,PRCP,SNWD,TAVG,TMAX,TMIN,granizo,latitude,longitude,...,om_rain_sum,om_snowfall_sum,om_precipitation_hours,om_wind_gusts_10m_max,om_wind_direction_10m_dominant,om_shortwave_radiation_sum,om_et0_fao_evapotranspiration,om_dew_point_2m_mean,om_relative_humidity_2m_mean,om_pressure_msl_mean
0,2000-01-01,"MALARGUE, AR",NaN,NaN,20.8,NaN,12.7,0,-35.48,-69.58,...,1.6,0.0,4.0,34.920000,217.959320,27.46,5.370969,10.142082,63.186707,1010.94586
1,2000-01-01,"MENDOZA AERO, AR",NaN,NaN,24.4,32.8,17.0,1,-32.83,-68.78,...,1.6,0.0,1.0,23.759998,10.304779,28.25,5.423854,16.552917,73.381630,1010.19995
2,2000-01-01,"SAN MARTIN, AR",0.0,NaN,24.1,NaN,17.5,0,-33.08,-68.48,...,0.1,0.0,1.0,27.000000,21.286467,28.25,5.550250,16.756416,70.660910,1009.64580
3,2000-01-01,"SAN RAFAEL, AR",3.0,NaN,22.3,31.0,15.2,0,-34.58,-68.33,...,3.6,0.0,3.0,32.039997,339.896600,28.97,6.062146,14.845166,62.548435,1009.40010
4,2000-01-02,"MALARGUE, AR",0.0,NaN,20.8,28.9,11.4,0,-35.48,-69.58,...,0.8,0.0,2.0,34.200000,327.639070,29.82,5.576764,10.317083,65.937706,1011.46660



--- Información del Dataset Final ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37399 entries, 0 to 37398
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            37399 non-null  object 
 1   station_name                    37399 non-null  object 
 2   PRCP                            9668 non-null   float64
 3   SNWD                            225 non-null    float64
 4   TAVG                            37397 non-null  float64
 5   TMAX                            19599 non-null  float64
 6   TMIN                            35148 non-null  float64
 7   granizo                         37399 non-null  int64  
 8   latitude                        37246 non-null  float64
 9   longitude                       37246 non-null  float64
 10  om_weather_code                 37246 non-null  float64
 11  om_temperature_2m_max           37246 non-null  float6